# Preprocesado
Este notebook carga los datos raw, los limpia, aplica feature engineering básico y avanzado, y guarda el CSV procesado.


In [ ]:
# 1. Imports y configuración
import pandas as pd
import numpy as np
from haversine import haversine
from sklearn.neighbors import BallTree
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import PolynomialFeatures

# Opcional: carga de config
# import yaml
# config = yaml.safe_load(open('config.yaml'))
# RAW_CSV = config['data']['raw_csv']
# PROC_CSV = config['data']['proc_csv']

RAW_CSV  = 'googlemaps/data/locales_todos.csv'
PROC_CSV = 'googlemaps/data/MAPS_locales_procesado.csv'

## 2. Carga y filtrado
- Eliminamos filas sin lat/lon  
- Conservamos solo negocios con ≥3 reseñas y puntuación existente


In [ ]:
df = pd.read_csv(RAW_CSV)
# Filtrado
df = df[df['latitud'].notna() & df['longitud'].notna()]
df = df[df['puntuacion_media'].notna() & (df['numero_reviews'] >= 3)]
df.shape, df.head()


## 3. Feature engineering básico
- Puntuación ponderada  
- Normalización Min–Max por categoría


In [ ]:
# 3.1 Puntuación ponderada
df['valoracion'] = df['puntuacion_media'] * (1 - np.exp(-df['numero_reviews'] / 10))

# 3.2 Normalización Min–Max por categoría
df['valoracion_norm'] = (
    df.groupby('categoria_negocio')['valoracion']
      .transform(lambda x: (x - x.min()) / (x.max() - x.min()))
)

df[['valoracion','valoracion_norm']].describe()


## 4. Feature engineering avanzado
1. Distancia al centro de la ciudad  
2. Densidades a varios radios (500 m, 1 km, 2 km) y ratio  
3. Clusterización espacial (DBSCAN)  
4. Transformaciones polinómicas e interacciones


In [ ]:
# 4.1 Distancia al centro (Madrid centro)
city_center = (40.4168, -3.7038)
df['dist_city_center_km'] = df.apply(
    lambda r: haversine((r.latitud, r.longitud), city_center), axis=1
)

# 4.2 Densidades
coords_rad = np.deg2rad(df[['latitud','longitud']].values)
tree = BallTree(coords_rad, metric='haversine')
for r_km in [0.5, 1.0, 2.0]:
    col = f'density_{int(r_km*1000)}m'
    df[col] = tree.query_radius(coords_rad, r=r_km/6371.0, count_only=True)
# Ratio competencia cercana vs amplia
df['ratio_500m_2km'] = df['density_500m'] / (df['density_2000m'] + 1)

# 4.3 Clusterización DBSCAN
db = DBSCAN(eps=0.5/6371.0, min_samples=10, metric='haversine')
df['cluster_zone'] = db.fit_predict(coords_rad).astype(str)

# 4.4 Polinomios e interacciones
poly_feats = ['dist_city_center_km', 'density_1000m']
poly = PolynomialFeatures(degree=2, include_bias=False)
arr = poly.fit_transform(df[poly_feats])
names = poly.get_feature_names_out(poly_feats)
poly_df = pd.DataFrame(arr, columns=names, index=df.index)
df = pd.concat([df, poly_df], axis=1)

# Vista rápida
df[['dist_city_center_km','density_500m','density_1000m','ratio_500m_2km','cluster_zone']+list(names)].head()


## 5. Guardado
Volcamos el DataFrame resultante a disco para usarlo en modelado.


In [ ]:
df.to_csv(PROC_CSV, index=False)
print("CSV procesado guardado en:", PROC_CSV)
